This code is the general case for the problem found in 5.3.4 of the textbook Power Flow Analysis by N. V. Ramana and uses the flowchart found in Fig 5.3 of the previous section to perform iterative Newton Raphson

**Imports**

In [1]:
import numpy as np
from numpy.linalg import inv

**Input Data**

In [2]:
# Bus data
# For each bus the data is as follows:
# 0:BusNum, 1:PG, 2:QG, 3:PD, 4:QD, 5:V, 6:PA, 7:BusType
# Where BusType 1 is Slack, 2 is PV, and 3 is PQ
bus_arr = np.array([[1, 0, 0, 0, 0, 1, 0, 1], 
                    [2, 1.63, 0, 0, 0, 1, 0, 2], 
                    [3, 0.85, 0, 0, 0, 1, 0, 2],
                    [4, 0, 0, 0, 0, 1, 0, 3],
                    [5, 0, 0, -0.90, -0.30, 1, 0, 3],
                    [6, 0, 0, 0, 0, 1, 0, 3],
                    [7, 0, 0, -1.00, -0.35, 1, 0, 3],
                    [8, 0, 0, 0, 0, 1, 0, 3],
                    [9, 0, 0, -1.25, -0.50, 1, 0, 3]])

# Reactive power limits
reactive_min = []
reactive_max = []
for i in bus_arr:
    if i[7] == 2:
        reactive_min.append(-3.00)
        reactive_max.append(3.00)
    else:
        reactive_min.append(0)
        reactive_max.append(0)
reactive_min = np.array(reactive_min)
reactive_max = np.array(reactive_max)

# Branch data
# For each branch the data is as follows:
# 0:BranchBuses, 1:LineImpedance
branch_arr = [[[1, 4], complex(0, 0.0576)],
              [[4, 5], complex(0.017, 0.092)],
              [[5, 6], complex(0.039, 0.17)],
              [[3, 6], complex(0, 0.0586)],
              [[6, 7], complex(0.0119, 0.1008)],
              [[7, 8], complex(0.0085, 0.072)],
              [[8, 2], complex(0, 0.0625)],
              [[8, 9], complex(0.032, 0.161)],
              [[9, 4], complex(0.01, 0.085)]]

**Function for creating Generator and Line Admittance arrays**

In [3]:
def create_GA_LA(Buses, Branches):
    # Impedance Values
    # Line Impedance Matrix
    LA = np.zeros((Buses.shape[0], Buses.shape[0]), dtype = complex)
    for b in Branches:
        # Branch b[0] contains a list of BusNums (BusNums are 1 higher than the index they are at)
        i = b[0][0]-1
        j = b[0][1]-1
        # Set LI for both buses to each other
        LA[i][j] = 1/b[1]
        LA[j][i] = 1/b[1]

    # Compute Generator Admittance
    GA = np.zeros(Buses.shape[0])
    
    return GA, LA

**Function for creating Y<sub>bus</sub>, G, and B**

In [4]:
def create_Ybus(Buses, GA, LA):
    Y = np.zeros((Buses.shape[0], Buses.shape[0]),  dtype = complex)
    for i in range(Y.shape[0]):
        Y[i][i] = Y[i][i] + GA[i]
        for j in range (Y.shape[1]):
            if i != j:
                Y[i, j] = Y[i, j] - LA[i][j]
                Y[i, i] = Y[i, i] + LA[i][j]
                
    G = Y.real
    B = Y.imag

    return Y, G, B

**Function for setting initial values of unknown quantities**

In [5]:
def init_unknown(V, PA):
    # Flat start for bus voltages
    # Unknown values for bus volage will be set to 1
    for i in range(V.size):
        if V[i] == 0:
            V[i] = 1

**Function for computing powers**

In [6]:
def compute_powers(Buses, V, PA, G, B):
    P_new = np.zeros(Buses.shape[0])
    Q_new = np.zeros(Buses.shape[0])
    # We don't need to calculate for the slack bus (the first bus), so we start at the second bus
    for i in range(1, Buses.shape[0]):
        for j in range (Buses.shape[0]):
            P_new[i] = P_new[i]+V[i]*V[j]*(G[i][j]*np.cos(PA[i]-PA[j])+B[i][j]*np.sin(PA[i]-PA[j]))
            Q_new[i] = Q_new[i]+V[i]*V[j]*(G[i][j]*np.sin(PA[i]-PA[j])-B[i][j]*np.cos(PA[i]-PA[j]))

    return P_new, Q_new

**Function for checking reactive power limits**

In [7]:
def check_limits(Buses, V, Q_new, Q_min, Q_max):
    for i in range(1, Buses.shape[0]):
        # Buses[i][7] is bus i's BusType, 2 is PV
        if Buses[i][7] == 2:
            # Check that Q for bus i is within limits
            if (Q_new[i] > Q_max[i]) or (Q_new[i] < Q_min[i]):
                # BusType 3 is PQ
                Buses[i][7] = 3
                # Set Bus's V to initial value
                Buses[i][5] = 1
                V[i] = 1
                return False
    # If the for loop is able to complete, all limit checks pass
    return True

**Function for computing power mismatches**

In [8]:
def compute_power_mismatches(Buses, P, Q, P_new, Q_new):
    Delta_P = []
    Delta_Q = []
    for i in range(1, Buses.shape[0]):
        Delta_P.append(P[i] - P_new[i])
        # Buses[i][7] is Bus i's BusType, 3 is PQ
        if Buses[i][7] == 3:
            Delta_Q.append(Q[i] - Q_new[i])
            
    
    return Delta_P, Delta_Q

**Function for computing the elements of the Jacobian matrix**

In [9]:
def compute_Jacobian(Buses, P_new, Q_new, V, PA, G, B):
    H = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
    N = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
    J = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
    L = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
    for i in range(1, Buses.shape[0]):
        for j in range(1, Buses.shape[0]):
            if i == j:
                H[i-1][j-1] = -Q_new[i]-B[i][i]*(V[i]**2)
                # Buses[i][7] is Bus i's BusType, 2 is PV
                if Buses[i][7] != 2:
                    N[i-1][j-1] = P_new[i]+G[i][i]*(V[i]**2)
                    J[i-1][j-1] = P_new[i]-G[i][i]*(V[i]**2)
                    L[i-1][j-1] = Q_new[i]-B[i][i]*(V[i]**2)
            else:
                H[i-1][j-1] = V[i]*V[j]*(G[i][j]*np.cos(PA[i]-PA[j])+B[i][j]*np.sin(PA[i]-PA[j]))
                if Buses[j][7] != 2:
                    N[i-1][j-1] = V[j]*V[i]*(G[i][j]*np.sin(PA[i]-PA[j])+B[i][j]*np.cos(PA[i]-PA[j]))
                if Buses[i][7] != 2:
                    J[i-1][j-1] = -V[i]*V[j]*(G[i][j]*np.cos(PA[i]-PA[j])+B[i][j]*np.sin(PA[i]-PA[j]))
                if Buses[i][7] != 2 and Buses[j][7] != 2:
                    L[i-1][j-1] = V[j]*V[i]*(G[i][j]*np.sin(PA[i]-PA[j])-B[i][j]*np.cos(PA[i]-PA[j]))

    for i in reversed(range(1, Buses.shape[0])):
        if Buses[i][7] == 2:
            # If Bus i is PV, remove column for that bus from N
            N = np.delete(N, i-1, 1)
            # If Bus i is PV, remove row for that bus from J
            J = np.delete(J, i-1, 0)
            # If Bus i is PV, remove row and column for that bus from L
            L = np.delete(L, i-1, 1)
            L = np.delete(L, i-1, 0)

    Jac = np.block([[H, N], [J, L]])
    return Jac

**Function for computing the increment matrix**

In [10]:
def compute_increment(Jac, Delta_P, Delta_Q):
    Delta_PQ = np.append(Delta_P, Delta_Q)
    Delta_PAV = np.dot(inv(Jac), Delta_PQ)
    Delta_PA = Delta_PAV[0:len(Delta_P)]
    Delta_V = Delta_PAV[len(Delta_P):]

    return Delta_PAV, Delta_PA, Delta_V

**Function for computing new values of $\delta$ and $V$**

In [11]:
def compute_new_PAV(Buses, V, PA, Delta_PA, Delta_V):
    PA_new = np.zeros(Buses.shape[0])
    V_new = np.zeros(Buses.shape[0])
    PA_new[0] = PA[0]
    V_new[0] = V[0]
    incr = 0
    for i in range(1, Buses.shape[0]):
        PA_new[i] = PA[i] + Delta_PA[i-1]
        # Buses[i][7] is Bus i's BusType, 3 is PQ
        if Buses[i][7] == 3:
            V_new[i] = V[i] + (Delta_V[incr])*V[i]
            incr = incr + 1
        else:
            V_new[i] = V[i]
    
    return PA_new, V_new

**Function for checking if $\delta$ and $V$ have converged**

In [12]:
def check_convergence(Buses, V, PA, V_new, PA_new, ep):
    convergence_PA = np.zeros(Buses.shape[0])
    convergence_V = np.zeros(Buses.shape[0])
    for i in range(Buses.shape[0]):
        convergence_PA[i] = abs(PA_new[i] - PA[i])
        convergence_V[i] = abs(V_new[i] - V[i])
    for i in range(Buses.shape[0]):
        if (convergence_PA[i] > ep):
            return False
        # Buses[i][7] is Bus i's BusType, 3 is PQ
        if Buses[i][7] == 3:
            if(convergence_V[i] > ep):
                return False
    # If this line is reached, all convergence levels are <= ep
    return True

**Function for loading calculated values into Bus array**

In [13]:
def load_values(Buses, oldBuses, V_new, PA_new):
    # Load calculated values into Buses and change altered PV buses back to PV
        for i in range(Buses.shape[0]):
            # Buses[i][6] is Bus i's PA
            Buses[i][6] = PA_new[i]
            # Buses[i][5] is Bus i's V
            Buses[i][5] = V_new[i]
            # Buses[i][7] is Bus i's BusType, 3 is PQ, 2 is PV
            if (Buses[i][7] == 3) and (oldBuses[i][7] == 2):
                Buses[i][7] = 2

**Function for calculating injected P and Q values**

In [14]:
def calculate_injected(Buses, G, B):
    for i in range(1, Buses.shape[0]):
        # Buses[i][7] is Bus i's BusType, 3 is PQ
        if Buses[i][7] == 2:
            for j in range(Buses.shape[0]):
                # Buses[i][2] is Bus i's QG, [i][5] is V, [i][6] is PA
                Buses[i][2] = Buses[i][2]+Buses[i][5]*Buses[j][5]*(G[i][j]*np.sin(Buses[i][6]-Buses[j][6])-B[i][j]*np.cos(Buses[i][6]-Buses[j][6]))

**Function for using Newton Raphson for Power Flow Problem**

In [15]:
def NR(oldBuses, Branches, Q_min, Q_max, ep):
    # Make a copy of the old buses to retain its data
    Buses = np.copy(oldBuses)

    # Load information on P, Q, V, and PA into variables
    P = []
    Q = []
    V = []
    PA = []
    for i in Buses:
        V.append(i[5])
        PA.append(i[6])
        if i[7] == 2:
            P.append(i[1])
            Q.append(i[2])
        else:
            P.append(i[3])
            Q.append(i[4])
    P = np.array(P)
    Q = np.array(Q)
    V = np.array(V)
    PA = np.array(PA)

    # Value calculations
    GA, LA = create_GA_LA(Buses, Branches)
    Y, G, B = create_Ybus(Buses, GA, LA)
    init_unknown(V, PA)

    total_iterations = 0 # Total iterations of NR
    iterations = 0 # Iterations since limit check last failed
    # Perform NR method iteratively
    # Stop condition handled within the loop
    it = 0
    while(True):
        P_new, Q_new = compute_powers(Buses, V, PA, G, B)
        # If limit check fails, treat ith PV bus as PQ bus and recompute powers
        if not check_limits(Buses, V, Q_new, Q_min, Q_max):
            iterations = 0
            P_new, Q_new = compute_powers(Buses, V, PA, G, B)
        Delta_P, Delta_Q = compute_power_mismatches(Buses, P, Q, P_new, Q_new)
        Jac = compute_Jacobian(Buses, P_new, Q_new, V, PA, G, B)
        Delta_PAV, Delta_PA, Delta_V = compute_increment(Jac, Delta_P, Delta_Q)
        PA_new, V_new = compute_new_PAV(Buses, V, PA, Delta_PA, Delta_V)
        total_iterations += 1
        iterations += 1
        # If convergence has been reached, exit loop
        if check_convergence(Buses, V, PA, V_new, PA_new, ep):
            break
        V = V_new
        PA = PA_new

        it += 1

    load_values(Buses, oldBuses, V_new, PA_new)    
    calculate_injected(Buses, G, B)

    # Print Data
    print("\nTotal nmber of iterations: ", total_iterations)
    print("Iterations since last limit check failure: ", iterations)
    print("\nOld Voltages:\n", oldBuses[:,5])
    print("\nNew Voltages:\n", Buses[:,5])    
    print("\nOld Phase Angles:\n", oldBuses[:,6])
    print("\nNew Phase Angles:\n", Buses[:,6])
    print("\nOld Bus Values:\n", oldBuses, "\n")
    print("New Bus Values (after NR):\n", Buses, "\n")

    return Buses

In [16]:
NR(bus_arr, branch_arr, reactive_min, reactive_max, 10**-2)


Total nmber of iterations:  6
Iterations since last limit check failure:  6

Old Voltages:
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]

New Voltages:
 [1.         1.         1.         0.95882269 0.93189107 0.97631141
 0.94983662 0.96916012 0.91554272]

Old Phase Angles:
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]

New Phase Angles:
 [ 0.          0.1641205   0.08434231 -0.03952745 -0.06641806  0.03554824
  0.00582116  0.06584373 -0.0761943 ]

Old Bus Values:
 [[ 1.    0.    0.    0.    0.    1.    0.    1.  ]
 [ 2.    1.63  0.    0.    0.    1.    0.    2.  ]
 [ 3.    0.85  0.    0.    0.    1.    0.    2.  ]
 [ 4.    0.    0.    0.    0.    1.    0.    3.  ]
 [ 5.    0.    0.   -0.9  -0.3   1.    0.    3.  ]
 [ 6.    0.    0.    0.    0.    1.    0.    3.  ]
 [ 7.    0.    0.   -1.   -0.35  1.    0.    3.  ]
 [ 8.    0.    0.    0.    0.    1.    0.    3.  ]
 [ 9.    0.    0.   -1.25 -0.5   1.    0.    3.  ]] 

New Bus Values (after NR):
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.0

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00],
       [ 2.00000000e+00,  1.63000000e+00,  5.68261584e-01,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.64120497e-01,  2.00000000e+00],
       [ 3.00000000e+00,  8.50000000e-01,  4.24071574e-01,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         8.43423084e-02,  2.00000000e+00],
       [ 4.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  9.58822688e-01,
        -3.95274539e-02,  3.00000000e+00],
       [ 5.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -9.00000000e-01, -3.00000000e-01,  9.31891069e-01,
        -6.64180582e-02,  3.00000000e+00],
       [ 6.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  9.76311406e-01,
         3.55482416e-02,  3.00000000e+00],
       [ 7.00000000e+00,  0.000000